<a href="https://colab.research.google.com/github/EddyDavies/Trade_With_Twitter/blob/main/ipynb/Transformer_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    from google.colab import drive
    IN_COLAB = True
    drive.mount('/content/drive')
except ModuleNotFoundError:
    IN_COLAB = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Check GPU

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Sep 12 23:12:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Check TPU

Check Memory

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
!git clone https://github.com/EddyDavies/Trade_With_Twitter.git

fatal: destination path 'Trade_With_Twitter' already exists and is not an empty directory.


In [5]:
!ls ../

bin	 datalab  home	 lib64	opt	    root  srv		     tmp    var
boot	 dev	  lib	 media	proc	    run   sys		     tools
content  etc	  lib32  mnt	python-apt  sbin  tensorflow-1.15.2  usr


In [6]:
%cd Trade_With_Twitter/src
!ls

/content/Trade_With_Twitter/src
bert_farm.ipynb  decorators.py	__pycache__  twitter
bert_farm.py	 prices		sentiment    utils.py


In [7]:
!pip install -r ../requirements.txt

In [8]:
import os
import pandas as pd
from tqdm import tqdm

from utils import get_date_array, get_month_array
from sentiment.inference import to_dict_of_lists, get_paths

In [9]:
# 10% before "2017-09-19" and 50% after
# dates_range = ("2017-01-01", "2021-06-20")
dates_range = ("2017-09-19", "2021-06-20")
months_range = ["Jan 17", "Jun 21"]

months = get_month_array(months_range)
dates = get_date_array(dates_range)

In [10]:
import torch

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [11]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, BertweetTokenizer

model_name = "finiteautomata/bertweet-base-sentiment-analysis"

tokenizer = BertweetTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
  
# tokenizer = tokenizer.to(device)
model = model.to(device)

sentiment_analysis = pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

In [12]:
if IN_COLAB:
    source, results_folder = get_paths(data_folder='../../drive/MyDrive/ColabData', model_name=model_name)
else:
    source, results_folder = get_paths(model_name=model_name)
results_folder

'../../drive/MyDrive/ColabData/bitcoin_scores/bertweet-base-sentiment-analysis_finiteautomata'

In [13]:
from sentiment.inference import get_tweets, get_sentiments, save_sentiments
from sentiment.utils import check_last_day

for date in dates:
    ids, tweets = get_tweets(date, source)

    percentage_per_chunk = 50
    save_every = 2000
    results = get_sentiments(date, tweets, ids, results_folder,
                              sentiment_analysis=sentiment_analysis,
                              save_every=save_every,
                              percentage_per_chunk=percentage_per_chunk)

    save_sentiments(ids, tweets, results_folder, date)
    check_last_day(results_folder, date)

FileNotFoundError: ignored

In [ ]:
!ls ../

In [ ]:
# !rm -r ../data/bitcoin_scores

In [ ]:
print(results_folder)

In [ ]:
!git pull